In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from datetime import datetime, date, time
import warnings
import random
warnings.filterwarnings('ignore')

Due to the original file is to large and the time computation is too long we have to split the original csv with the objective to be able to work with a shorter dataset. We made a random split cause is important to get robust results and the data is ordered by date and it isportant to have data form all of the dates to analyse the dirvers stops during a determinate period of years.

We decided that from the big dataset that has more than 11 milions of rows (11 milions of drivers stops data) works only with 500000 that is aproximately an 4,5% of the original dataset.

In [4]:
#Loading our dataset splited in a random way:
filename = "Desktop/m_3/wa.csv"
n = sum(1 for line in open(filename)) - 1 #number of records in file (excludes header)
s = 500000 #desired sample size
skip = sorted(random.sample(range(1,n+1),n-s)) #the 0-indexed header will not be included in the skip list
wa = pd.read_csv(filename, skiprows=skip)

In [5]:
wa

,raw_row_number,date,time,location,lat,lng,county_name,subject_age,subject_race,subject_sex,...,frisk_performed,search_conducted,search_basis,raw_officer_race,raw_officer_gender,raw_contact_type,raw_driver_race,raw_driver_gender,raw_search_type,raw_enforcements
0,52,2009-10-31,09:00:00,S-009-1,47.796712,-122.143463,Snohomish County,42.0,white,female,...,False,False,NaN,WHITE,-,--,White,F,N,-|-
1,83,2009-10-05,10:00:00,S-167-21,47.395457,-122.232636,King County,31.0,black,male,...,False,False,NaN,WHITE,-,--,African American,M,N,-|-
2,88,2009-10-03,15:00:00,I-005-84,46.727501,-122.977152,Lewis County,30.0,white,male,...,False,False,NaN,WHITE,-,--,White,M,N,-
3,118,2009-10-28,07:00:00,I-205-32,45.683082,-122.593698,Clark County,24.0,white,male,...,False,False,NaN,HISPANIC,-,--,White,M,N,-|-
4,138,2009-10-02,00:00:00,NaN,NaN,NaN,NaN,22.0,white,male,...,False,False,NaN,WHITE,-,--,White,M,N,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,11961087,2018-09-08,10:00:00,I-082-91,46.263809,-119.535174,Benton County,NaN,NaN,NaN,...,False,False,NaN,NaN,-,-,NaN,NaN,NaN,NaN
499996,11961100,2018-09-20,14:00:00,I-082-58,46.345853,-120.177747,Yakima County,18.0,hispanic,female,...,False,False,NaN,NaN,-,-,Hispanic,F,N,-
499997,11961151,2018-09-11,13:00:00,S-CMP-40,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,NaN,NaN,-,-,NaN,NaN,NaN,-
499998,11961154,2018-09-17,13:00:00,S-CMP-40,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,NaN,NaN,-,-,NaN,NaN,NaN,-


In [6]:
#Pandas dataframe.notnull() function detects existing/ non-missing values in the dataframe, we will use to filter the data useful:
wa = wa[wa.notnull()]
wa

,raw_row_number,date,time,location,lat,lng,county_name,subject_age,subject_race,subject_sex,...,frisk_performed,search_conducted,search_basis,raw_officer_race,raw_officer_gender,raw_contact_type,raw_driver_race,raw_driver_gender,raw_search_type,raw_enforcements
0,52,2009-10-31,09:00:00,S-009-1,47.796712,-122.143463,Snohomish County,42.0,white,female,...,False,False,NaN,WHITE,-,--,White,F,N,-|-
1,83,2009-10-05,10:00:00,S-167-21,47.395457,-122.232636,King County,31.0,black,male,...,False,False,NaN,WHITE,-,--,African American,M,N,-|-
2,88,2009-10-03,15:00:00,I-005-84,46.727501,-122.977152,Lewis County,30.0,white,male,...,False,False,NaN,WHITE,-,--,White,M,N,-
3,118,2009-10-28,07:00:00,I-205-32,45.683082,-122.593698,Clark County,24.0,white,male,...,False,False,NaN,HISPANIC,-,--,White,M,N,-|-
4,138,2009-10-02,00:00:00,NaN,NaN,NaN,NaN,22.0,white,male,...,False,False,NaN,WHITE,-,--,White,M,N,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,11961087,2018-09-08,10:00:00,I-082-91,46.263809,-119.535174,Benton County,NaN,NaN,NaN,...,False,False,NaN,NaN,-,-,NaN,NaN,NaN,NaN
499996,11961100,2018-09-20,14:00:00,I-082-58,46.345853,-120.177747,Yakima County,18.0,hispanic,female,...,False,False,NaN,NaN,-,-,Hispanic,F,N,-
499997,11961151,2018-09-11,13:00:00,S-CMP-40,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,NaN,NaN,-,-,NaN,NaN,NaN,-
499998,11961154,2018-09-17,13:00:00,S-CMP-40,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,NaN,NaN,-,-,NaN,NaN,NaN,-


Before to continuing it is interesting to see the diferent data types for each column that we have in our main dataframe.

In [7]:
wa.dtypes

raw_row_number          int64
date                   object
time                   object
location               object
lat                   float64
lng                   float64
county_name            object
subject_age           float64
subject_race           object
subject_sex            object
officer_race           object
officer_sex            object
department_name        object
type                   object
violation              object
arrest_made            object
citation_issued          bool
warning_issued           bool
outcome                object
contraband_found       object
frisk_performed          bool
search_conducted         bool
search_basis           object
raw_officer_race       object
raw_officer_gender     object
raw_contact_type       object
raw_driver_race        object
raw_driver_gender      object
raw_search_type        object
raw_enforcements       object
dtype: object

In [8]:
#It is important for the followings parts to get the columns of date and time into a datetime type.
#There ara many ways to do this tranformation of the data type but we decide to use the pandas function pd.to_datetime:

#First for the date column:
wa_datetime = pd.to_datetime(wa.date)
wa.date = wa_datetime

Before continuing with the task of hangling data, he have to comprovate that we get in a good way the datetime step because is esential for the next parts of the project.

In [9]:
wa.head()

,raw_row_number,date,time,location,lat,lng,county_name,subject_age,subject_race,subject_sex,...,frisk_performed,search_conducted,search_basis,raw_officer_race,raw_officer_gender,raw_contact_type,raw_driver_race,raw_driver_gender,raw_search_type,raw_enforcements
0,52,2009-10-31,09:00:00,S-009-1,47.796712,-122.143463,Snohomish County,42.0,white,female,...,False,False,NaN,WHITE,-,--,White,F,N,-|-
1,83,2009-10-05,10:00:00,S-167-21,47.395457,-122.232636,King County,31.0,black,male,...,False,False,NaN,WHITE,-,--,African American,M,N,-|-
2,88,2009-10-03,15:00:00,I-005-84,46.727501,-122.977152,Lewis County,30.0,white,male,...,False,False,NaN,WHITE,-,--,White,M,N,-
3,118,2009-10-28,07:00:00,I-205-32,45.683082,-122.593698,Clark County,24.0,white,male,...,False,False,NaN,HISPANIC,-,--,White,M,N,-|-
4,138,2009-10-02,00:00:00,NaN,NaN,NaN,NaN,22.0,white,male,...,False,False,NaN,WHITE,-,--,White,M,N,-


In [10]:
wa.dtypes

raw_row_number                 int64
date                  datetime64[ns]
time                          object
location                      object
lat                          float64
lng                          float64
county_name                   object
subject_age                  float64
subject_race                  object
subject_sex                   object
officer_race                  object
officer_sex                   object
department_name               object
type                          object
violation                     object
arrest_made                   object
citation_issued                 bool
warning_issued                  bool
outcome                       object
contraband_found              object
frisk_performed                 bool
search_conducted                bool
search_basis                  object
raw_officer_race              object
raw_officer_gender            object
raw_contact_type              object
raw_driver_race               object
r

We have to create a new column to see if the driver stop is after or before the legalitzation of the recreative marijuana in Washignton. If is before, we will get a 1 in the column and a 0 if is after this day.

In [11]:
#Creating a variable with this information to compare after with our dataframe values.
a = '2012-12-06'
a_datetime = pd.to_datetime(a)
a = a_datetime
a

Timestamp('2012-12-06 00:00:00')

To create this new column we will use the tool date.toordinal cause is easier for python works with this type of data and also is easier to make comparation. We will pass to this new format the date column creating a new column with a new name date_ordinal. Also we will do the same with the date that marijuana becomes legal in Washignton due to after we will use this date to make an analysis and create new columns with useful information for the final model.

In [12]:
#Creating the new column with the date_ordinal format:
wa['date_ordinal'] = wa['date'].apply(lambda date: date.toordinal())

#Convert the date of marijuana legalization in a date_odinal format:
a_ordinal = a.toordinal()

We have to check if we get the format that we want.

In [13]:
wa.date_ordinal

0         733711
1         733685
2         733683
3         733708
4         733682
           ...  
499995    736945
499996    736957
499997    736948
499998    736954
499999    736963
Name: date_ordinal, Length: 500000, dtype: int64

In [14]:
a_ordinal

734843

How we can see we get the values in a more manageable format to make operations. We will create the column of pre_legalization.

In [15]:
wa['pre_legalization'] = wa.date_ordinal.apply(lambda x: x < a_ordinal)
wa.pre_legalization = wa.pre_legalization.astype(int)
wa.pre_legalization

0         1
1         1
2         1
3         1
4         1
         ..
499995    0
499996    0
499997    0
499998    0
499999    0
Name: pre_legalization, Length: 500000, dtype: int32

How we can see we get this new column that we will use later, but the next step is calculate the time in years after legalitzation for each stop that happens in this concret period. For the other values that we will obtain during the period before legalitzation we will put a 0 value to represent this fact.

In [16]:
#We will call this column like years_after_legalization:
wa['years_after_legalization'] = (wa['date_ordinal']-a_ordinal)/365
wa.years_after_legalization

0        -3.101370
1        -3.172603
2        -3.178082
3        -3.109589
4        -3.180822
            ...   
499995    5.758904
499996    5.791781
499997    5.767123
499998    5.783562
499999    5.808219
Name: years_after_legalization, Length: 500000, dtype: float64

Now we have to put in a better way, where all the values under 0 will be zero to get a more clean column.

In [18]:
#Creating the length range subrtacting one due to the index in the loop begins with i=0.
length = wa.shape[0]
l=range(length-1)
l

range(0, 499999)

In [19]:
for i in l:
    
    b = wa.years_after_legalization[i]
    
    if b < 0:
        b = 0
        wa.years_after_legalization[i] = b

Finally, the last step in this section will be to see if the columns with the expected values have been obtained.

In [21]:
wa

,raw_row_number,date,time,location,lat,lng,county_name,subject_age,subject_race,subject_sex,...,raw_officer_race,raw_officer_gender,raw_contact_type,raw_driver_race,raw_driver_gender,raw_search_type,raw_enforcements,date_ordinal,pre_legalization,years_after_legalization
0,52,2009-10-31,09:00:00,S-009-1,47.796712,-122.143463,Snohomish County,42.0,white,female,...,WHITE,-,--,White,F,N,-|-,733711,1,0.000000
1,83,2009-10-05,10:00:00,S-167-21,47.395457,-122.232636,King County,31.0,black,male,...,WHITE,-,--,African American,M,N,-|-,733685,1,0.000000
2,88,2009-10-03,15:00:00,I-005-84,46.727501,-122.977152,Lewis County,30.0,white,male,...,WHITE,-,--,White,M,N,-,733683,1,0.000000
3,118,2009-10-28,07:00:00,I-205-32,45.683082,-122.593698,Clark County,24.0,white,male,...,HISPANIC,-,--,White,M,N,-|-,733708,1,0.000000
4,138,2009-10-02,00:00:00,NaN,NaN,NaN,NaN,22.0,white,male,...,WHITE,-,--,White,M,N,-,733682,1,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,11961087,2018-09-08,10:00:00,I-082-91,46.263809,-119.535174,Benton County,NaN,NaN,NaN,...,NaN,-,-,NaN,NaN,NaN,NaN,736945,0,5.758904
499996,11961100,2018-09-20,14:00:00,I-082-58,46.345853,-120.177747,Yakima County,18.0,hispanic,female,...,NaN,-,-,Hispanic,F,N,-,736957,0,5.791781
499997,11961151,2018-09-11,13:00:00,S-CMP-40,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-,-,NaN,NaN,NaN,-,736948,0,5.767123
499998,11961154,2018-09-17,13:00:00,S-CMP-40,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-,-,NaN,NaN,NaN,-,736954,0,5.783562


We can show that the columns obtained are in a correct way like we expected and rigth now we areable to begin with the analysis.